# IBM Data Science Capstone Project: *Battle of Neighbourhoods*

**The idea behind the project:** 
Define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute.

**Background:** Bart is living in Warsaw, Poland, and currently looking to move to some other city in Europe that is as comfortable as Warsaw but with higher income. 
<br/>
**Idea:** Build a model that will classify and group european cities to help Bart narrow the list of the cities to choose from.

P.S.: *Bart knows that I am studying Data Science and asked to help him with this question.*

**Data**

To perform this analysis I will need list of largest European cities, their geolocation and GDP per capita.

* List of European largest cities (euro_cities.csv) + population, from: https://ec.europa.eu/eurostat/databrowser/view/urb_cpop1/default/table?lang=en
* Geolocation will be pulled using geopy library and foresquare
* City attractions: foresquare - list of venues
* GDP per city (gdp.csv): https://ec.europa.eu/eurostat

In [274]:
#Web scraping
import requests
from bs4 import BeautifulSoup as bs4

#Data manipulation
import pandas as pd
import numpy as np

#Visualization
import folium
from matplotlib import pyplot as plt
import seaborn as sns

#Geocords
from geopy.geocoders import Nominatim

### Data collection and preprocessing

The base of the data model will be list of european cities with total population from 2017 to 2019 year. The rest of data will be tied up to this main data drame.

In [380]:
cities = pd.read_csv('data/euro_cities.csv',encoding='utf8')
cities.columns = ['city','2017','2018','2019']
cities.head()

,city,2017,2018,2019
0,Belgium,11351727,11398589,11455519
1,Bruxelles / Brussel,1199095,1205492,1215289
2,Antwerpen,520859,523591,526439
3,Gent,259462,260329,262205
4,Charleroi,201884,202341,202637


As you might notice first element "Belgium" - country. The following are the cities of Belgium. Lets create an additional column where we would classify conventions.

In [381]:
#get list of countries
countries = pd.read_csv('data/countries.csv',encoding='utf8')
countries = list(countries['Country'])

isCountry = [True if x in countries else False for x in cities.city]
cities['isCountry'] = isCountry
cities.head()

,city,2017,2018,2019,isCountry
0,Belgium,11351727,11398589,11455519,True
1,Bruxelles / Brussel,1199095,1205492,1215289,False
2,Antwerpen,520859,523591,526439,False
3,Gent,259462,260329,262205,False
4,Charleroi,201884,202341,202637,False


Lets create a separate column to track countries for cities

In [382]:
cities.loc[cities.isCountry==True,'country'] = cities.loc[cities.isCountry==True,'city']
cities.country.fillna(method='ffill', inplace=True)
cities.head()

,city,2017,2018,2019,isCountry,country
0,Belgium,11351727,11398589,11455519,True,Belgium
1,Bruxelles / Brussel,1199095,1205492,1215289,False,Belgium
2,Antwerpen,520859,523591,526439,False,Belgium
3,Gent,259462,260329,262205,False,Belgium
4,Charleroi,201884,202341,202637,False,Belgium


We actually do not need all three year of statistics on populations, we just need the latest data. Hence, lets populate empty records for 2019 with records from previous years.

In [383]:
cities.replace(':',np.nan,inplace=True)
cities['2019'].isnull().sum()

572

In [384]:
cities.loc[cities['2019'].isnull(),'2019'] = cities.loc[cities['2019'].isnull(),'2018']
cities.loc[cities['2019'].isnull(),'2019'] = cities.loc[cities['2019'].isnull(),'2017']
#cities['2019'].isna().sum()
#Dropping the rest
cities = cities.loc[cities['2019'].notnull(),:]
print(cities['2019'].isna().sum(),cities.shape)

0 (844, 6)


And finally dropping the rest 2017, 2018 columns

In [385]:
cities.drop(['2017','2018'], axis=1, inplace=True)
cities.rename(columns={'2019':'population'},inplace=True)
print(cities.shape)
cities.head()

(844, 4)


,city,population,isCountry,country
0,Belgium,11455519,True,Belgium
1,Bruxelles / Brussel,1215289,False,Belgium
2,Antwerpen,526439,False,Belgium
3,Gent,262205,False,Belgium
4,Charleroi,202637,False,Belgium


##### Processing GDP Data

Now as we have List of cities collected with its population, lets add GDP the table.

In [386]:
gdp = pd.read_csv('data/gdp_euro.csv')
gdp.drop(['UNIT'],axis=1,inplace=True)
gdp.columns = ['city','2017','2018','2019']
print(gdp.shape)
gdp.head()

(346, 4)


,city,2017,2018,2019
0,Belgium,430372.10,446364.90,459819.80
1,Bruxelles / Brussel,134293.37,137974.75,141689.31
2,Antwerpen,48646.35,50478.13,52137.95
3,Gent,27470.15,28830.02,29916.27
4,Charleroi,11384.32,11689.12,11682.73


Lets check GDP for empty value and clear them if any.

In [387]:
gdp.replace(':',np.nan,inplace=True)
gdp['2019'].isnull().sum()

242

In [388]:
gdp.loc[gdp['2019'].isnull(),'2019'] = gdp.loc[gdp['2019'].isnull(),'2018']
gdp.loc[gdp['2019'].isnull(),'2019'] = gdp.loc[gdp['2019'].isnull(),'2017']
#cities['2019'].isna().sum()
#Dropping the rest
gdp = gdp.loc[gdp['2019'].notnull(),:]
print(gdp['2019'].isna().sum(),gdp.shape)

0 (344, 4)


Now lets remove previous years and update column names

In [389]:
gdp.drop(['2017','2018'], axis=1, inplace=True)
gdp.rename(columns={'2019':'gdp'},inplace=True)
print(gdp.shape)
gdp.head()

(344, 2)


,city,gdp
0,Belgium,459819.80
1,Bruxelles / Brussel,141689.31
2,Antwerpen,52137.95
3,Gent,29916.27
4,Charleroi,11682.73


Also GDP data contains some 'Non-metropolitan area' estimations for some countries, removing these. And lets remove country figueres.

In [390]:
gdp = gdp[~gdp.city.str.contains('Non-metro')]
gdp.shape

(316, 2)

Lets add out GDP figures to cities data frame

In [391]:
cities = pd.merge(cities, gdp, on='city')

In [392]:
print(cities[cities.isCountry==False].shape)
cities[cities.isCountry==False].head()

(226, 5)


,city,population,isCountry,country,gdp
1,Bruxelles / Brussel,1215289,False,Belgium,141689.31
2,Antwerpen,526439,False,Belgium,52137.95
3,Gent,262205,False,Belgium,29916.27
4,Charleroi,202637,False,Belgium,11682.73
5,Liège,383482,False,Belgium,21195.70


Removing double city names and diplicated rows if any.

In [393]:
cities.loc[cities.city.str.contains('/'),'city'] = [x.split('/')[0].strip() for x in cities.loc[cities.city.str.contains('/'),'city']]
cities.city.replace('([(|)])','',regex=True, inplace=True)
cities.drop_duplicates(subset='city', keep='last', inplace=True)
cities.head()

,city,population,isCountry,country,gdp
0,Belgium,11455519,True,Belgium,459819.80
1,Bruxelles,1215289,False,Belgium,141689.31
2,Antwerpen,526439,False,Belgium,52137.95
3,Gent,262205,False,Belgium,29916.27
4,Charleroi,202637,False,Belgium,11682.73


From my experience geopy is providing bad quality data, especially for europe. Therefore, I used Foreqsquare venue API to find places **nearby**, this endpoint retuns as well coordinated of the cities.

Catching coordinates by city name.

In [397]:
CLIENT_ID = 'Y4SPYFQGWXXVGSNSI3M5PARWTNYUBHWCHXQYD2BJA5VP3PFA' # your Foursquare ID
CLIENT_SECRET = 'U4TXUQUZFRDVPRKBF4TL5RS5VQTDKUU4QNEE2ZFHG2AZW0YN' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4SPYFQGWXXVGSNSI3M5PARWTNYUBHWCHXQYD2BJA5VP3PFA
CLIENT_SECRET:U4TXUQUZFRDVPRKBF4TL5RS5VQTDKUU4QNEE2ZFHG2AZW0YN


In [398]:
long = []
lat = []

radius = 5000
offset = 0
LIMIT = 1

for index, row in cities.iterrows():
    try:
        near = row['city']
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}&offset={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                near, 
                radius, 
                LIMIT,
                offset)
        results = requests.get(url).json()["response"]['geocode']['center']
        lat.append(results['lat'])
        long.append(results['lng'])
    except:
        lat.append(None)
        long.append(None)

cities['latitude_frsq'] = lat
cities['longitude_frsq'] = long
cities = cities[~cities.latitude_frsq.isnull()]
cities.head()

,city,population,isCountry,country,gdp,latitude,longitude,latitude_frsq,longitude_frsq
0,Belgium,11455519,True,Belgium,459819.80,50.640281,4.666715,50.75000,4.50000
1,Bruxelles,1215289,False,Belgium,141689.31,50.846557,4.351697,50.85045,4.34878
2,Antwerpen,526439,False,Belgium,52137.95,51.221110,4.399708,51.21989,4.40346
3,Gent,262205,False,Belgium,29916.27,51.053829,3.725012,51.05000,3.71667
4,Charleroi,202637,False,Belgium,11682.73,50.412033,4.443624,50.41136,4.44448


Getting Europe location

In [400]:
location = geolocator.geocode('Europe')
print('Europe geolocation: {}, {}'.format(location.latitude, location.longitude))
latitude = location.latitude
longitude = location.longitude

Europe geolocation: 51.0, 10.0


So far so good, lets see our cities on tha map

In [401]:
cities = cities[(cities['longitude']<38) & (cities['longitude']>-14)]

# create map of Manhattan using latitude and longitude values
europe_map = folium.Map(location=[latitude, longitude], zoom_start=4)

# add markers to map
for lat, lng, label in zip(cities[cities.isCountry==False]['latitude_frsq'], cities[cities.isCountry==False]['longitude_frsq'], cities[cities.isCountry==False]['city']):
    label = folium.Popup(label + f' {lat}, {lng}', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(europe_map)  
    
europe_map

Now as we have coordinated lets gather foresquare data on top venues in city. Since cities are of different size lets consider only center city attractions in the 5 km radius.

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [402]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Lets utilize the function from lab to get list of venues for a certain adress

In [370]:
city_latitude = cities['latitude_frsq'] #  latitude value
city_longitude = cities['longitude_frsq'] #  longitude value
city_name = cities['city'] # city name

In [409]:
LIMIT = 100
radius=5000

names, latitudes, longitudes = city_name, city_latitude, city_longitude

venues_list=[]
for name, lat, lng in zip(names, latitudes, longitudes):
    print(name, end=' ')

    # create the API request URL
    offset =0
    for i in range(5):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            offset)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        offset += len(results)
    print(offset, end=' ')
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['city', 
              'city latitude', 
              'city longitude', 
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

Warszawa 240 Kraków 237 Gdansk 184 

Now lets see how many venues we have per city.

In [431]:
city_venues = nearby_venues
city_venues.shape

(28947, 7)

In [432]:
print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))

There are 550 uniques categories.


In [433]:
city_venues.groupby('city').count()

,city latitude,city longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
city,,,,,,
A Coruña,117,117,117,117,117,117
Aachen,173,173,173,173,173,173
Amiens,70,70,70,70,70,70
Angers,69,69,69,69,69,69
Antwerpen,234,234,234,234,234,234
...,...,...,...,...,...,...
Wuppertal,86,86,86,86,86,86
Würzburg,113,113,113,113,113,113
Zagreb,187,187,187,187,187,187


#### Analyze each city

In [429]:
# one hot encoding
cities_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cities_onehot['city'] = city_venues['city']


# move neighborhood column to the first column
fixed_columns = ['city'] + [x for x in cities_onehot.columns.tolist() if x!='city']
cities_onehot = cities_onehot[fixed_columns]

cities_onehot.head()

,city,ATM,Abruzzo Restaurant,Acai House,Accessories Store,Adult Education Center,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Bruxelles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bruxelles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bruxelles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bruxelles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bruxelles,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [434]:
cities_onehot.shape

(28947, 551)

#### Now lets get averages for city based on the venue category

In [435]:
cities_grouped = cities_onehot.groupby('city').mean().reset_index()
cities_grouped

,city,ATM,Abruzzo Restaurant,Acai House,Accessories Store,Adult Education Center,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,A Coruña,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.034188,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,Aachen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.005780,0.000000,0.0,0.0,0.000000,0.0,0.0,0.005780,0.000000
2,Amiens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.014286,0.000000
3,Angers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.014493,0.014493,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Antwerpen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.004274,0.000000,0.0,0.0,0.004274,0.0,0.0,0.004274,0.004274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Wuppertal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
199,Würzburg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.026549,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
200,Zagreb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.010695,0.000000,0.0,0.0,0.000000,0.0,0.0,0.005348,0.000000
201,Zaragoza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000


#### Now lets create a new dataframe for top vanues in cities. However, number of venue categories is pretty large hence lets stick with 30 top venues.

First, let's write a function to sort the venues in descending order.


In [436]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Build a new DF

In [442]:
num_top_venues = 30

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    indx = str(ind+1)[-1]
    if indx == '1':
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[0]))
    elif indx == '2':
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[1]))
    elif indx == '3':
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[2]))
    else:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['city'] = cities_grouped['city']

for ind in np.arange(cities_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cities_grouped.iloc[ind, :], num_top_venues)

print(cities_venues_sorted.shape)
cities_venues_sorted.head()

(203, 31)


,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,21st Most Common Venue,22nd Most Common Venue,23rd Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue
0,A Coruña,Spanish Restaurant,Restaurant,Beach,Park,Plaza,Tapas Restaurant,Hotel,Wine Bar,Seafood Restaurant,...,Garden,Mediterranean Restaurant,Beer Bar,Food,Tea Room,Cupcake Shop,Big Box Store,Bookstore,Sushi Restaurant,Theater
1,Aachen,Supermarket,Café,Italian Restaurant,German Restaurant,Bakery,Bar,Drugstore,Plaza,Greek Restaurant,...,Gastropub,Pub,Outlet Store,Discount Store,Mediterranean Restaurant,Chinese Restaurant,Cocktail Bar,Snack Place,Scenic Lookout,Shopping Mall
2,Amiens,Supermarket,Bar,Hotel,French Restaurant,Fast Food Restaurant,Shopping Mall,Park,Plaza,Performing Arts Venue,...,Brewery,Gas Station,Golf Course,Convenience Store,Cosmetics Shop,Café,Burger Joint,Sporting Goods Shop,Pizza Place,Soccer Stadium
3,Angers,Bar,Park,French Restaurant,Supermarket,Hotel,Tram Station,Restaurant,Beer Garden,Shopping Mall,...,Japanese Restaurant,Miscellaneous Shop,Opera House,Farmers Market,Movie Theater,Museum,Music Venue,Brewery,Nightclub,Trail
4,Antwerpen,Bar,Coffee Shop,Italian Restaurant,French Restaurant,Cocktail Bar,Bakery,Gym,Park,Supermarket,...,Breakfast Spot,Steakhouse,Fast Food Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,Sandwich Place,Thai Restaurant,Concert Hall,Juice Bar


And finally lets add our GDP and population to table.

In [446]:
cities_venues_sorted = pd.merge(cities_venues_sorted, cities[['city','population','gdp']], on='city')
cities_venues_sorted.shape

(202, 33)

GDP and population are not accurate representation of economic level of city. Therefore, lets calculate GDP per Capita to have common measure to compare cities. 

In [456]:
cities_venues_sorted.gdp = cities_venues_sorted.gdp.astype('float')
cities_venues_sorted.population = cities_venues_sorted.population.astype('float')

In [457]:
cities_venues_sorted['GdpPerCap'] = cities_venues_sorted.gdp / cities_venues_sorted.population
cities_venues_sorted.drop(['gdp','population'], axis=1, inplace=True)
cities_venues_sorted.head()

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,22nd Most Common Venue,23rd Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,GdpPerCap
0,A Coruña,Spanish Restaurant,Restaurant,Beach,Park,Plaza,Tapas Restaurant,Hotel,Wine Bar,Seafood Restaurant,...,Mediterranean Restaurant,Beer Bar,Food,Tea Room,Cupcake Shop,Big Box Store,Bookstore,Sushi Restaurant,Theater,0.106404
1,Aachen,Supermarket,Café,Italian Restaurant,German Restaurant,Bakery,Bar,Drugstore,Plaza,Greek Restaurant,...,Pub,Outlet Store,Discount Store,Mediterranean Restaurant,Chinese Restaurant,Cocktail Bar,Snack Place,Scenic Lookout,Shopping Mall,0.083495
2,Amiens,Supermarket,Bar,Hotel,French Restaurant,Fast Food Restaurant,Shopping Mall,Park,Plaza,Performing Arts Venue,...,Gas Station,Golf Course,Convenience Store,Cosmetics Shop,Café,Burger Joint,Sporting Goods Shop,Pizza Place,Soccer Stadium,0.109353
3,Angers,Bar,Park,French Restaurant,Supermarket,Hotel,Tram Station,Restaurant,Beer Garden,Shopping Mall,...,Miscellaneous Shop,Opera House,Farmers Market,Movie Theater,Museum,Music Venue,Brewery,Nightclub,Trail,0.120917
4,Antwerpen,Bar,Coffee Shop,Italian Restaurant,French Restaurant,Cocktail Bar,Bakery,Gym,Park,Supermarket,...,Steakhouse,Fast Food Restaurant,Pizza Place,Cosmetics Shop,Gym / Fitness Center,Sandwich Place,Thai Restaurant,Concert Hall,Juice Bar,0.099039
